## Evaluate

$ 5x^{3} - 4x^{2}y^{2} + 13xy^{2} + x^{2} -10y = out $

$ v_1 = xx $

$ v_2 = yy $

$ v_3 = 5xv_1 $

$ v_4 = 4v_1v_2 $

$ out -v_3 + v_4 - v_1 + 10y = 13xv_2 $

$ w = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4
\end{bmatrix} $



$ L = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 5 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 \\
0 & 0 & 13 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} $


$ R = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
\end{bmatrix} $


$ O = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 10 & -1 & 0 & -1 & 1 \\
\end{bmatrix} $


In [ ]:
import galois
import numpy as np

p = 71
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


In [ ]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''L
{Lp}
''')

print(f'''R
{Rp}
''')

print(f'''O
{Op}
''')

In [ ]:
def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( galois.Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

def evaluate_poly(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

alpha = FP(2)
beta = FP(3)
tau = FP(20)
gamma = FP(5)
delta = FP(7)

# U = to_poly(Lp)
# print_poly("U", U)

# V = to_poly(Rp)
# print_poly("V", V)

# W = to_poly(Op)
# print_poly("W", W)

# U_beta = [ poly * beta for poly in U ]
# print_poly("βU", U_beta)

# V_alpha = [ poly * alpha for poly in V ]
# print_poly("αV", V_alpha)

# bUaUW = np.add(np.add(U_beta, V_alpha), W)
# print_poly("(βU + αV + W) = K", bUaUW)

# K = evaluate_poly(bUaUW, tau)
# print_evaluation("(βU + αV + W) = K", K)

T = galois.Poly(FP([1, p-1]))
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly(FP([1, p-i]))

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

In [ ]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d-1]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree - 1)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd-1(τ) + αVd-1(τ) + Wd-1(τ)]
# cterms_G1 = [multiply(G1, int(k)) for k in K]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-2 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 2)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree - 1)]

In [ ]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

In [ ]:
hCoeff = H.coefficients()[::-1]

# h0 * G1[τ^0 * T(τ)], h1 * G1[τ^1 * T(τ)], ..., hd-2 * G1[τ^d-2 * T(τ)]
HT_tems = [multiply(point, int(coeff)) for point, coeff in zip(target_G1, hCoeff)]

# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = HT_tems[0]
for i in range(1, len(HT_tems)):
    HT_G1 = add(HT_G1, HT_tems[i])

print("HT_G1 = ", normalize(HT_G1))

uCoef = U.coefficients()[::-1]

# u0 * G1[τ^0], u1 * G1[τ^1], ..., ud-1 * G1[τ^d-1]
U_tems = [multiply(point, int(coeff)) for point, coeff in zip(tau_G1, uCoef)]

# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = U_tems[0]
for i in range(1, len(U_tems)):
    A_G1 = add(A_G1, U_tems[i])

print("A_G1 = ", normalize(A_G1))

vCoef = V.coefficients()[::-1]

# v0 * G2[τ^0], v1 * G2[τ^1], ..., vd-1 * G2[τ^d-1]
V_tems = [multiply(point, int(coeff)) for point, coeff in zip(tau_G2, vCoef)]

# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = V_tems[0]
for i in range(1, len(V_tems)):
    B_G2 = add(B_G2, V_tems[i])

print("B_G2 = ", normalize(B_G2))

wCoef = W.coefficients()[::-1]

# w0 * G1[τ^0], w1 * G1[τ^1], ..., wd-1 * G1[τ^d-1]
W_tems = [multiply(point, int(coeff)) for point, coeff in zip(tau_G1, wCoef)]

# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
C_G1 = W_tems[0]
for i in range(1, len(W_tems)):
    C_G1 = add(C_G1, W_tems[i])

print("C_G1 = ", normalize(C_G1))

C_G1 = add(C_G1, HT_G1)

pairing(B_G2, A_G1) == pairing(G2, C_G1)